In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import streamlit as st
import os

for key in st.secrets.keys():
    os.environ[key] = st.secrets[key]

from load_llm import openai_llm, groq_llm, llm, gemini_llm
from langchain_core.prompts import ChatPromptTemplate

# llm = gemini_llm

I0000 00:00:1724215781.156371  409748 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [5]:
res = gemini_llm.invoke("what is langgraph?")
res.pretty_print()

================================== Ai Message ==================================

LangGraph is a **knowledge graph** specifically designed for **natural language processing (NLP)**. It's a powerful tool that helps computers understand and reason about language in a more human-like way. 

Here's a breakdown of its key features and how it works:

**What it is:**

* **A graph database:** LangGraph stores information in a network of interconnected nodes (entities) and edges (relationships). This structure allows for efficient querying and reasoning about complex relationships between words, concepts, and other linguistic elements.
* **Knowledge-based:** It contains a vast amount of pre-existing knowledge about language, including:
    * **Lexical information:** Word meanings, synonyms, antonyms, etc.
    * **Syntactic information:** Grammar rules, part-of-speech tags, etc.
    * **Semantic information:** Relationships between concepts, hierarchies, etc.
    * **World knowledge:** General f

In [6]:
from typing import *
from langchain_core.pydantic_v1 import BaseModel, Field


class Supervisor(BaseModel):
    """select agent and explantion for selecting the agent"""

    agent: Literal["ai_expert", "stock_trader", "generic"] = Field(
        ...,
        description="""select following agents based user question agents: ai_expert, stock_trader, generic""",
    )

    explanation: str = Field(description="explain the reason for selecting specific agent")

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            "You are a supervisor tasked with selecting right agent to act next 
            
            ai_expert
            Expertise:  AI, Data science
            Description: An ai_researcher is an expert in researching and developing new machine learning and AI models, algorithms, and techniques. They have in-depth knowledge of data science and can collect, analyze, and interpret complex data to inform their research.

            stock_trader
            Expertise: Stock market analysis, trading
            Description: A stock_trader is an expert in analyzing and predicting stock market trends, and making informed trading decisions. They have in-depth knowledge of financial markets, economic indicators, and trading strategies.

            generic
            Expertise: General topics
            
            Description: If a topic does not fall under the specific expertise of the other agents, the query will default to the generic agent. This ensures that a wide array of questions can be addressed, even if they do not pertain to the specialized fields of the other agents.

            specify reason for selecting the agent
            """,
        ),
        ("human", "{query}"),
    ]
)

structured_llm = openai_llm.with_structured_output(Supervisor)

structured_llm = prompt | structured_llm

In [8]:
query = "How to learn spring framework"

res = structured_llm.invoke({"query": query})
print(res)

agent='ai_expert' explanation="The user is asking about learning the Spring framework, which is a Java-based framework for building enterprise-level applications. The ai_expert agent is selected because it has expertise in AI and Data science, and learning a framework like Spring requires a strong foundation in programming and software development, which is within the ai_expert's domain."


In [80]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information. Give explanation why you are using a tool",
        ),
        ("placeholder", "{chat_history}"),
        ("placeholder", "{agent_scratchpad}"),
        ("human", "{input}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True
)

In [28]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
res = agent_executor.invoke({"input": "what is LangChain?"})

In [34]:
from load_llm import openai_llm, groq_llm
from langchain_core.prompts import ChatPromptTemplate

from typing import *
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import *


class Planner(BaseModel):
    """create outline for answering the query"""

    steps: str = Field(description=""" create outline for answering the query""")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
As a planner,  create outline for answering the query
            Previous conversation history:

            {chat_history}
            """,
        ),
        ("human", "{query}"),
    ]
)

structured_llm = groq_llm.with_structured_output(Planner)

structured_llm = prompt | structured_llm


# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
query = """why osama bin laden destroyed twin towers """
response = structured_llm.invoke({"query": query, "chat_history": []})
print(response.steps)

1. Identify the motivations behind Osama bin Laden's actions
2. Discuss the events leading up to the 9/11 attacks
3. Explain the role of Al-Qaeda in the attacks
4. Describe the impact of the attacks on the United States and the world
5. Provide context on the aftermath of the attacks and the response from the international community


In [24]:
query = """why osama bin laden destroyed twin towers """
response = structured_llm.invoke({"query": query, "chat_history": []})

plan_str = response.steps
print(plan_str)

AttributeError: 'NoneType' object has no attribute 'steps'

In [17]:
print(response.response_style_prompt)

Provide a detailed explanation of the events and motivations behind the destruction of the Twin Towers by Osama bin Laden.


In [81]:
input = f"""
For below user query
{query} 
Use below points while responding: 
{plan_str}
                      """

res = agent_executor.invoke({"input": input})
from IPython.display import display, Markdown, Latex

display(Markdown(res["output"]))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Osama bin Laden motivations for 9/11 attacks'}`


[{'url': 'https://en.wikipedia.org/wiki/Motives_for_the_September_11_attacks', 'content': 'In an essay, Doran argued that bin Laden\'s followers "consider themselves an island of true believers surrounded by a sea of iniquity".[65] Doran further argued that bin Laden hoped that US retaliation would unite the faithful against the West, sparking revolutions in Arab nations and elsewhere, and that the Osama bin Laden videos were attempting to provoke a visceral reaction in the Middle East aimed at a violent reaction by Muslim citizens to increased US involvement in their region.[66]\nBergen argued that the attacks were part of a plan to cause the United States to increase its military and cultural presence in the Middle East, thereby forcing Muslims to confront the idea of a non-Muslim government and establish conservative Islamic governments i

Osama bin Laden destroyed the Twin Towers as part of a larger plan to inflict harm on the United States and its interests. The motivations behind his actions were rooted in his extremist ideology and a desire to defend Islam against perceived Western aggression.

Bin Laden was motivated by a range of factors, including:

1. Opposition to US foreign policy in the Middle East, particularly the presence of US troops in Saudi Arabia.
2. Anger over US support for Israel and its treatment of Palestinians.
3. A desire to unite Muslims against the West and establish a global Islamic caliphate.

The events leading up to the 9/11 attacks were complex and multifaceted. They included:

1. The Soviet-Afghan War, which drew bin Laden and other jihadists to Afghanistan to fight against the Soviet Union.
2. The Gulf War, which led to the presence of US troops in Saudi Arabia and further radicalized bin Laden.
3. The rise of Al-Qaeda, which bin Laden founded in the late 1980s and which became a global network of jihadists.

Al-Qaeda played a central role in the 9/11 attacks, which were planned and carried out by a group of operatives led by bin Laden. The attacks were intended to inflict maximum damage on the United States and its economy, and to create a sense of fear and uncertainty among the American people.

The impact of the 9/11 attacks on global politics and international relations was significant. They led to:

1. The launch of the War on Terror, a global military campaign aimed at rooting out Al-Qaeda and other jihadist groups.
2. The invasion of Afghanistan, which toppled the Taliban regime and led to a long and bloody conflict.
3. The invasion of Iraq, which was justified by the US government as part of the War on Terror but was widely criticized as a mistake.

The consequences of the 9/11 attacks on the United States and the world were far-reaching. They included:

1. The loss of thousands of lives, both on the day of the attacks and in the subsequent wars in Afghanistan and Iraq.
2. A significant increase in military spending and a shift towards a more militarized foreign policy.
3. A rise in Islamophobia and anti-Muslim sentiment in the United States and other Western countries.
4. A decline in civil liberties and an increase in surveillance and security measures.

Overall, the 9/11 attacks were a pivotal moment in modern history, with far-reaching consequences for global politics, international relations, and the lives of millions of people around the world.

In [69]:
response = groq_llm.invoke(
    f"""
For below user query
{query} 
Use below points while responding: 
{plan_str}

                      """
)
from IPython.display import display, Markdown, Latex

display(Markdown(response.content))

The 2008 market crash, also known as the Global Financial Crisis (GFC), was a complex and multifaceted event with various contributing factors. Here's a breakdown of the key events and factors that led to the crisis:

**Key Events Leading Up to the 2008 Market Crash:**

1. **Housing Market Boom (2000-2006)**: The US housing market experienced a significant boom, fueled by low interest rates, lax lending standards, and excessive speculation.
2. **Subprime Mortgage Crisis (2006-2007)**: Many subprime borrowers, who were not able to afford mortgages, defaulted on their loans, causing a surge in foreclosures and a subsequent decline in housing prices.
3. **Financial Deregulation (1999-2004)**: The Gramm-Leach-Bliley Act (1999) repealed parts of the Glass-Steagall Act, allowing commercial banks to engage in investment activities, increasing their exposure to risk. The Securities and Exchange Commission (SEC) also relaxed regulations, enabling financial institutions to take on more leverage.
4. **Excessive Leverage and Risk-Taking (2004-2007)**: Financial institutions, including banks and investment banks, took on excessive leverage and risk, creating complex financial instruments, such as mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).

**Role of Subprime Mortgage Crisis:**

The subprime mortgage crisis played a significant role in the 2008 market crash. Many subprime borrowers were unable to afford their mortgages, leading to a wave of defaults and foreclosures. This caused a decline in housing prices, which in turn led to a decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs) held by financial institutions.

**Impact of Housing Market Bubble Burst:**

The housing market bubble burst in 2006-2007, leading to a sharp decline in housing prices. This decline in housing prices reduced the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs), causing significant losses for financial institutions.

**Effect of Financial Deregulation:**

Financial deregulation contributed to the 2008 market crash by allowing financial institutions to take on excessive leverage and risk. The Gramm-Leach-Bliley Act (1999) and the SEC's relaxation of regulations enabled commercial banks to engage in investment activities, increasing their exposure to risk.

**Consequences of Excessive Leverage and Risk-Taking:**

Excessive leverage and risk-taking by financial institutions led to a significant increase in their exposure to risk. When the housing market bubble burst, many financial institutions found themselves with large amounts of worthless mortgage-backed securities (MBS) and collateralized debt obligations (CDOs), leading to significant losses and eventual collapse.

**Timeline of Major Events During the Crisis:**

* August 2007: French bank BNP Paribas halts withdrawals from its investment funds, citing subprime mortgage exposure.
* March 2008: Bear Stearns, a US investment bank, is sold to JPMorgan Chase after facing financial difficulties.
* September 7, 2008: Fannie Mae and Freddie Mac, US government-sponsored mortgage giants, are placed into conservatorship.
* September 14, 2008: Merrill Lynch, a US investment bank, is sold to Bank of America.
* September 15, 2008: Lehman Brothers, a US investment bank, files for bankruptcy.
* October 3, 2008: The US government passes the Troubled Asset Relief Program (TARP) to provide $700 billion in bailout funds to struggling financial institutions.

**Key Factors That Contributed to the 2008 Market Crash:**

1. **Subprime Mortgage Crisis**: Widespread defaults on subprime mortgages led to a decline in housing prices and a subsequent decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).
2. **Financial Deregulation**: The Gramm-Leach-Bliley Act (1999) and the SEC's relaxation of regulations enabled financial institutions to take on excessive leverage and risk.
3. **Excessive Leverage and Risk-Taking**: Financial institutions took on excessive leverage and risk, creating complex financial instruments that ultimately led to significant losses.
4. **Housing Market Bubble Burst**: The housing market bubble burst in 2006-2007, leading to a sharp decline in housing prices and a subsequent decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).

In summary, the 2008 market crash was caused by a combination of factors, including the subprime mortgage crisis, financial deregulation, excessive leverage and risk-taking, and the housing market bubble burst. These factors created a perfect storm that ultimately led to the collapse of several major financial institutions and a global economic downturn.

In [89]:
from langchain_community.retrievers import TavilySearchAPIRetriever

tavily_retriever = TavilySearchAPIRetriever(k=2)


def get_results_from_tavily(search_queries):
    queries_docs = tavily_retriever.batch(search_queries)
    docs = [{"href": doc.metadata["source"], "content": doc.page_content} for docs in queries_docs for doc in docs]
    return docs

In [90]:
def search_web(search_queries: str, explanation: str) -> str:
    """The search_web tool allows to augment knowledge and retrieve the latest information on a topic by searching the web using provided comma-separated search queries max queries 2. This tool returns a summary of the key points from all retrieved web pages, enabling me to provide more accurate and up-to-date information.

    Args:
        search_queries (str): comma seperated search queries without quotes, related to the topic, max queries 3
        explanation (str): give a reason for choosing this tool

    Returns:
        str: summary of all web pages retrieved for a search query
    """

    search_queries = search_queries.split(",")
    # search_queries = [clean_text(query) for query in search_queries]

    urls = []
    results = get_results_from_tavily(search_queries)
    urls += [item["href"] for item in results]

    print("****   WEB SEARCH  *****")
    print(search_queries)
    for url in urls:
        print(url)

    summaries = "\n\n".join([f"\n\nContent: {(doc['content'])}" for idx, doc in enumerate(results)])
    return summaries

In [96]:
print(search_web("elon musk wikipedia, jeff bezos", ""))

****   WEB SEARCH  *****
['elon musk wikipedia', ' jeff bezos']
https://www.wikiwand.com/en/Elon_Musk
https://simple.wikipedia.org/wiki/Elon_Musk
https://www.britannica.com/money/Jeff-Bezos
https://en.wikipedia.org/wiki/Jeff_Bezos


Content: He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023[update], according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]
A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and presiden

In [ ]:
from chat_graph import graph

In [2]:
import streamlit as st
import os

for key in st.secrets.keys():
    os.environ[key] = st.secrets[key]

from load_llm import openai_llm, groq_llm, llm, gemini_llm
from langchain_core.prompts import ChatPromptTemplate

# llm = gemini_llm


from chat_graph import graph

# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))


import time
from IPython.display import display, Markdown, Latex


def display_response(text):
    display(Markdown(text))

I0000 00:00:1724254858.566688  907556 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [3]:
config = {"configurable": {"thread_id": int(time.time())}}

prompt = " explain how LLM works "
response = graph.invoke({"messages": [("user", prompt)]}, config)
# print(response)

***** Planner *****
1. To explain how Large Language Models (LLM) work, I'll break down the process into a series of steps. Here's a detailed outline:

**Step 1: Data Collection**

* Identify a massive dataset of text from various sources, such as books, articles, research papers, and websites.
* The dataset should be diverse, covering a wide range of topics, styles, and genres.
* The data is typically collected and preprocessed by a team of experts to ensure its quality and relevance.

**Step 2: Data Preprocessing**

* Clean and preprocess the collected data by removing unnecessary characters, punctuation, and special symbols.
* Tokenize the text into individual words or subwords (smaller units of words).
* Convert the text into a numerical representation using techniques such as word embeddings (e.g., Word2Vec, GloVe) or one-hot encoding.

**Step 3: Model Architecture**

* Design a neural network architecture that can process and learn from the preprocessed data.
* Common architectur

In [4]:
display_response(response["messages"][-1].content)

**How Large Language Models (LLMs) Work: A Step-by-Step Explanation**

Large Language Models (LLMs) have revolutionized the field of natural language processing (NLP) by enabling machines to understand, generate, and interact with human language. To explain how LLMs work, I'll break down the process into a series of steps, as outlined below.

**Step 1: Data Collection**

The first step in building an LLM is to collect a massive dataset of text from various sources, such as books, articles, research papers, and websites. This dataset should be diverse, covering a wide range of topics, styles, and genres. The data is typically collected and preprocessed by a team of experts to ensure its quality and relevance.

According to a study by Radford et al. (2019), the dataset used to train the popular language model BERT consists of 3.3 billion parameters and 16 GB of text data [1]. The dataset includes a mix of books, articles, and websites, which helps the model learn to recognize patterns and relationships in language.

**Step 2: Data Preprocessing**

Once the dataset is collected, it needs to be cleaned and preprocessed to remove unnecessary characters, punctuation, and special symbols. This step is crucial in preparing the data for the model to learn from.

Tokenization is another important step in data preprocessing, where the text is broken down into individual words or subwords (smaller units of words). This helps the model to understand the relationships between words and learn to predict the next word in a sequence.

Word embeddings, such as Word2Vec and GloVe, are used to convert the text into a numerical representation. This allows the model to learn the semantic meaning of words and their relationships with each other.

**Step 3: Model Architecture**

The next step is to design a neural network architecture that can process and learn from the preprocessed data. Common architectures for LLMs include Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, and Transformers.

RNNs are well-suited for sequential data, such as text, and can learn to recognize patterns and relationships in language. LSTMs are a type of RNN that can handle long-term dependencies in data.

Transformers, on the other hand, are a type of neural network that uses self-attention mechanisms to process sequential data. They have been shown to be highly effective in language modeling tasks and are widely used in LLMs.

**Step 4: Model Training**

Once the model architecture is designed, it needs to be trained on the preprocessed dataset using a suitable optimization algorithm, such as stochastic gradient descent or Adam.

The model learns to predict the next word or token in a sequence, given the context of the previous words. This is done by minimizing a loss function, such as cross-entropy or mean squared error.

The training process involves multiple iterations, with the model adjusting its parameters to minimize the loss function. This process can take several days or even weeks, depending on the size of the dataset and the complexity of the model.

**Step 5: Model Evaluation**

After the model is trained, it needs to be evaluated on a separate test dataset to assess its performance. Metrics such as perplexity, accuracy, and F1-score are used to evaluate the model's ability to predict the next word or token.

Perplexity is a measure of how well the model predicts the next word in a sequence. A lower perplexity score indicates better performance.

Accuracy and F1-score are measures of how well the model performs on specific tasks, such as language translation or sentiment analysis.

**Step 6: Model Fine-Tuning**

Once the model is trained and evaluated, it can be fine-tuned on a specific task or domain to adapt it to the task requirements. This involves retraining the model on a smaller dataset related to the task, while keeping the pre-trained weights as a starting point.

Fine-tuning allows the model to learn task-specific patterns and relationships, which can improve its performance on the task.

**Step 7: Model Deployment**

After fine-tuning, the model can be deployed in a production environment, where it can be used to generate text, answer questions, or perform other tasks.

The model can be integrated with other systems, such as chatbots, virtual assistants, or content generation platforms.

**Step 8: Model Maintenance and Updates**

Finally, the model needs to be continuously monitored and updated to maintain its accuracy and relevance. New data can be added to the model to keep it up-to-date with the latest developments and trends.

The model can be retrained or fine-tuned to adapt to changes in the task requirements or domain.

In conclusion, LLMs are complex systems that require careful design, training, and evaluation to achieve good performance. By following the steps outlined above, LLMs can be developed and deployed to perform a wide range of tasks, from language translation and text summarization to question answering and content generation.

References:

[1] Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2019). Improving Language Understanding by Generative Models. arXiv preprint arXiv:1907.07655.

[2] Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. arXiv preprint arXiv:1908.08504.

[3] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention Is All You Need. arXiv preprint arXiv:1706.03762.

[4] Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., ... & Amodei, D. (2020). Language Models are Few-Shot Learners. arXiv preprint arXiv:2005.14165.

In [13]:
# config = {"configurable": {"thread_id": int(time.time())}}

prompt = " generate cover story for upwork with experience in deep learning"
response = graph.stream({"messages": [("user", prompt)]}, stream_mode="values")
# print(response)

In [12]:
for chunk in response:
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

 generate cover story for upwork with experience in deep learning
***** Planner *****
1. 1. Identify relevant work experience in deep learning, 2. Highlight key skills and accomplishments, 3. Emphasize transferable skills, 4. Create a compelling personal statement, 5. Develop a professional profile, 6. Prepare a portfolio of work, 7. Research and tailor the proposal to the client, 8. Write a clear and concise proposal, 9. Include relevant certifications and education, 10. Edit and proofread the proposal for grammar and spelling errors
================================ Human Message =================================

 generate cover story for upwork with experience in deep learning
****   WEB SEARCH  *****
['upwork cover letter deep learning experience']
https://community.upwork.com/t5/Freelancers/Crafting-the-Perfect-Upwork-Cover-Letter-A-Comprehensive-Guide/m-p/1653151
https://ebizfacts.com/upwork-cover-l

Fetching pages: 100%|##########| 5/5 [00:01<00:00,  3.01it/s]


================================== Ai Message ==================================

Based on the provided content and steps, here's a sample cover letter for Upwork with experience in deep learning:

Dear [Client's Name],

I'm excited to apply for your deep learning project. With over 5 years of experience in deep learning, I have the skills and expertise to deliver high-quality results.

**Skills Highlight:**

* Proficient in TensorFlow and PyTorch
* Experience with convolutional neural networks (CNNs) and recurrent neural networks (RNNs)
* Strong understanding of transfer learning and fine-tuning pre-trained models

**Relevant Experience:**

* Developed a deep learning model for image classification that achieved an accuracy of 95% on the CIFAR-10 dataset.
* Built a natural language processing (NLP) model that improved sentiment analysis by 20% on a benchmark dataset.

**Call to Action:**

I would love to discuss your project in more detail. Please feel free to reach out to schedule a 